# Here goes the title
---

Some description.

*Code made by [You](https://github.com/@you) - YYYY.*

## Importing Standard Libraries

In [2]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

### Import Scripts

Here you can import any python file that is on the `scripts/` folder.

In [3]:
import example

Running example.py ...
Running other_module.py ...
Done!
hi!
Done!


# One Dimenstion Likelihood

The likelihood is the joint probability distribution of a random sample for a fixed observation. We want to know what is the best parameters $\theta$ that models the given data $x$ given a statistical model with a probability density function $f$.

$$L(\theta | x) = f(x | \theta)$$

For the case of $x \sim N(\mu, \sigma)$, then 

$$f(x | \mu, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{(x - \mu)^2}{2 \sigma^2})$$

if we have more than one observation, $X = x_1, ... , x_n$, the joint probability distribution is,

$$f(X | \mu, \sigma) = \prod^n_{i=1} \frac{1}{\sqrt{2\pi\sigma^2}}exp(-\frac{(x_i - \mu)^2}{2 \sigma^2})$$

usually is easier to work with the logarithm of it since it preserves the concavity of exponential funcitons, so

$$ln\;f(X | \mu, \sigma) = -\frac{n}{2}ln(2\pi) - \frac{n}{2}ln(\sigma^2)-\frac{1}{2 \sigma^2}\sum^n_{i=1}(x_i - \mu)^2$$

which is the likelihood.

We can find the parameter pair that best fits a fixed sample by findind the maximum of the joint distribution.

Given that the normal has two parameters, we can hold on constant to find the maximum. And, invert the process to find the other parameter maximum.

Let's start by defining the vector $X$

In [5]:
X = [1, 2, 3, 4, 5]

Now, we have to define a function for the normal given $x_i, \mu$ and $\sigma$

In [11]:
def normal(x_i, mu, sigma):
    
    return 1 / np.sqrt(2 * np.pi * sigma * sigma) * \
            np.exp( - np.power(x_i - mu, 2) / (2 * sigma * sigma))   

Let's check if makes sense by plotting it

In [51]:
x = np.arange(-3, 5, 0.1)

y = normal(x, 1, 1)

pd.DataFrame(y, x).iplot()

It looks like a normal curve. Nice!

The likelihood is just passing our vector $X$ through the normal probability density and multiplying the results. But, we are still able to vary the parameters $\mu$ and $\sigma$.

Now, let's see the likelihood plot if we fix $\sigma = 1$ and vary $\mu$

In [56]:
sigma = 1
mus = np.arange(-0, 6, 0.1)

# there must be a vectorized way to do that, but this is fast
y = []
for mu in mus:
    y_temp = 1
    for x_i in X:
        y_temp = y_temp * normal(x_i, mu, sigma)
    y.append(y_temp)
        
pd.DataFrame(y, mus).iplot(xTitle='$\\mu$', yTitle='Likelihood $L(\\mu | X)$',
                          title='Likelihood ')


On the other hand, if we fix $\mu = 1$ and vary $\sigma$

In [60]:
sigmas = np.arange(-0, 6, 0.1)
mu = 1

# there must be a vectorized way to do that, but this is fast
y = []
for sigma in sigmas:
    y_temp = 1
    for x_i in X:
        y_temp = y_temp * normal(x_i, mu, sigma)
    y.append(y_temp)
        
pd.DataFrame(y, sigmas).iplot(xTitle='$\\sigma$', yTitle='Likelihood $L(\\sigma | X)$',
                          title='Likelihood ')

In both plots, the maximum likelihood is where the parameter is most probable. Very simple, we can try to plot both $\sigma$ and $\mu$ at the same time just for fun.

In [64]:
sigmas = np.arange(-0, 6, 0.1)
mus = np.arange(-3, 6, 0.1)

# there must be a vectorized way to do that, but this is fast
z = []
for mu in mus:
    for sigma in sigmas:
        z_temp = 1
        for x_i in X:
            z_temp = z_temp * normal(x_i, mu, sigma)
        z.append(z_temp)
        

In [71]:
a = np.asarray(z)
a.shape = (len(mus), len(sigmas))

In [82]:

fig = go.Figure(data=[go.Surface(z=a)], 
               layout={'xaxis': {'title': '$\mu'}}
               )

fig.iplot()


In [81]:
fig['layout']

Layout({
    'xaxis': {'title': {'text': '$\\mu'}}
})